# Constraining the inversion

This notebook explains some additional approaches we employ to `smallness regularization`. In this scenario, we aim to invert for the sediment-basement contact with some outcropping basement at the surface. We employ the following constraints:
1) Inverted basement should not cross the surface topography
2) Inverted basement should not go below 10 km
3) Inverted basement shouldn't change at locations where it is outcropping. 

Again, we will use the same synthetic data from the past examples.

## Import packages

In [ ]:
%load_ext autoreload
%autoreload 2
import os

import pandas as pd
import verde as vd
import xarray as xr
from polartoolkit import maps
from polartoolkit import utils as polar_utils

import invert4geom

os.environ["POLARTOOLKIT_HEMISPHERE"] = "south"

## Create observed gravity data

In [ ]:
true_surface_topography, _, _, _ = invert4geom.load_synthetic_model(
    spacing=1000,
    region=(0, 40000, 0, 30000),
    zref=0,
)

# create lower synthetic topography data
true_basement_topography = invert4geom.synthetic_topography_regional(
    spacing=1000,
    region=(0, 40000, 0, 30000),
    scale=3,
    yoffset=300,
)

# clip lower topography to be always below true topography
true_basement_topography = xr.where(
    true_basement_topography < true_surface_topography,
    true_basement_topography,
    true_surface_topography,
)
cpt_lims = polar_utils.get_combined_min_max(
    [true_surface_topography, true_basement_topography]
)
fig = maps.plot_grd(
    true_surface_topography,
    fig_height=10,
    title="Surface topography",
    cmap="rain",
    reverse_cpt=True,
    cpt_lims=cpt_lims,
    hist=True,
    cbar_label="elevation (m)",
    frame=["nSWe", "xaf10000", "yaf10000"],
)

fig = maps.plot_grd(
    true_basement_topography,
    fig=fig,
    origin_shift="x",
    fig_height=10,
    title="Basement topography",
    cmap="rain",
    reverse_cpt=True,
    cpt_lims=cpt_lims,
    hist=True,
    cbar_label="elevation (m)",
    frame=["nSwE", "xaf10000", "yaf10000"],
)
fig.show()

In [ ]:
outcropping_basement = true_basement_topography.where(
    true_basement_topography == true_surface_topography,
)
outcropping_basement.plot()

In [ ]:
# # the first density contrast is between sediment (~1800 kg/m3) and air (~1 kg/m3)
# density_contrast_1 = 1800 - 1

# # the second density contrast is between basement (~2800 kg/m3) and air
# # (~1 kg/m3)
# density_contrast_2 = 2800 - 1

# # use it to create a surface density distribution
# surface_density_contrast = xr.where(
#     outcropping_basement.isnull(),
#     density_contrast_1,
#     density_contrast_2,
# )
surface_density_contrast = 1800 - 1

surface_model = invert4geom.create_model(
    zref=true_surface_topography.to_numpy().mean(),
    density_contrast=surface_density_contrast,
    starting_topography=true_surface_topography.to_dataset(name="upward"),
)

surface_model.inv.plot_model(
    color_by="density",
    backend="static",
    zscale=30,
)

In [ ]:
# # the first density contrast is between basement (~2800 kg/m3) and sediment (~1800 kg/m3)
# density_contrast_1 = 2800 - 1800

# # the second density contrast is between basement (~2800 kg/m3) and air (~1 kg/m3)
# # (~1 kg/m3)
# density_contrast_2 = 2800 - 1

# # create a basement density distribution
# basement_density_contrast = xr.where(
#     outcropping_basement.isnull(),
#     density_contrast_1,
#     density_contrast_2,
# )
basement_density_contrast = 2800 - 1800

basement_model = invert4geom.create_model(
    zref=true_basement_topography.to_numpy().mean(),
    # zref = 0,
    density_contrast=basement_density_contrast,
    starting_topography=true_basement_topography.to_dataset(name="upward"),
)

basement_model.inv.plot_model(
    color_by="density",
    backend="static",
    zscale=30,
)

In [ ]:
invert4geom.plot_prism_layers(
    [surface_model, basement_model],
    # color_by="constant",
    color_by="density",
    zscale=30,
    constant_colors=["saddlebrown", "blue"],
    # azimuth=115,
    backend="static",
)

In [ ]:
# make pandas dataframe of locations to calculate gravity
# this represents the station locations of a gravity survey
# create lists of coordinates
coords = vd.grid_coordinates(
    spacing=1000,
    region=(0, 40000, 0, 30000),
    pixel_register=False,
    extra_coords=1001,  # survey elevation
)

# grid the coordinates
observations = vd.make_xarray_grid(
    (coords[0], coords[1]),
    data=coords[2],
    data_names="upward",
    dims=("northing", "easting"),
)

grav_data = invert4geom.create_data(observations)

# forward gravity of upper and lower prisms
grav_data.inv.forward_gravity(surface_model, "surface_grav")
grav_data.inv.forward_gravity(basement_model, "basement_grav")

grav_data["gravity_anomaly"] = grav_data.surface_grav + grav_data.basement_grav
grav_data

In [ ]:
fig = maps.plot_grd(
    grav_data.surface_grav,
    fig_height=10,
    title="surface prisms gravity",
    cmap="viridis",
    hist=True,
    cbar_label="mGal",
    frame=["nSWe", "xaf10000", "yaf10000"],
)

fig = maps.plot_grd(
    grav_data.basement_grav,
    fig=fig,
    origin_shift="x",
    fig_height=10,
    title="basement prisms gravity",
    cmap="viridis",
    hist=True,
    cbar_label="mGal",
    frame=["nSwE", "xaf10000", "yaf10000"],
)

fig = maps.plot_grd(
    grav_data.gravity_anomaly,
    fig=fig,
    origin_shift="x",
    fig_height=10,
    title="Observed gravity",
    cmap="viridis",
    hist=True,
    cbar_label="mGal",
    frame=["nSwE", "xaf10000", "yaf10000"],
)

fig.show()

In [ ]:
# create 10 random point within the outcropping basement region
num_constraints = 10
coords = vd.scatter_points(
    region=vd.pad_region(grav_data.region, -500),
    size=num_constraints,
    random_state=0,
)
constraint_points = pd.DataFrame(data={"easting": coords[0], "northing": coords[1]})

# sample basement topography at these points
constraint_points = invert4geom.sample_grids(
    constraint_points,
    true_basement_topography,
    "upward",
)
constraint_points.head()

## Create starting model
Create a starting model from the interpolation of the 10 constraint points. Also use each grid cell of outcropping basement as an additional constraint point so the starting model equals the basement where it outcrops.

During interpolation, use `upper_confining_layer` and `lower_confining_layer` to make the starting model doesn't ever go above the surface, or below a flat layer at 100 m elevation. 

In [ ]:
# add all points where basement outcrops as constraint points
df = outcropping_basement.to_dataframe().reset_index().dropna()

# set these outcropping points of "outside" the area to interpolate
df["inside"] = False

# set the constraint points as within the area to interpolate
constraint_points["inside"] = True

# merge buffer and inside points
constraint_points = pd.concat((df, constraint_points))
constraint_points

In [ ]:
# grid the sampled values using verde
starting_topography_kwargs = dict(
    method="splines",
    region=basement_model.region,
    spacing=basement_model.spacing,
    constraints_df=constraint_points,
    dampings=None,
    upper_confining_layer=true_surface_topography,
)

starting_topography = invert4geom.create_topography(**starting_topography_kwargs)

_ = polar_utils.grd_compare(
    true_basement_topography,
    starting_topography,
    grid1_name="True topography",
    grid2_name="Starting topography",
    robust=True,
    hist=True,
    inset=False,
    verbose="q",
    title="difference",
    grounding_line=False,
    reverse_cpt=True,
    cmap="rain",
    points=constraint_points,
    points_style="x.3c",
)

Add a `mask` variable to the starting topography denote where outcropping basement is located with a value of 0, and other locations with a value of 1. Cells with a mask value of 0 won't be altered during the inversion. 

In [ ]:
ds = starting_topography.to_dataset(name="upward")
# ds["mask"] = xr.where(
#     abs(ds.upward -true_surface_topography)<1, 0, 1
# )
# ds.mask.plot()

In [ ]:
model = invert4geom.create_model(
    zref=true_basement_topography.to_numpy().mean(),
    density_contrast=2800 - 1800,
    starting_topography=ds,
    upper_confining_layer=true_surface_topography,
)
model.inv.plot_model(
    color_by="density",
    zscale=20,
    backend="static",
)

## Gravity misfit
Now we need to calculate the forward gravity of the starting topography. We then can subtract it from our observed gravity to get a starting gravity misfit.

In [ ]:
grav_data.inv.forward_gravity(model)

grav_data.inv.regional_separation(
    method="constant",
    constant=0,
    # constraints_df=constraint_points,
)
grav_data

In [ ]:
grav_data.inv.plot_anomalies()

## Inversion

In [ ]:
# setup the inversion
inv = invert4geom.Inversion(
    grav_data,
    model,
    solver_damping=0.01,
    # set stopping criteria
    max_iterations=30,
    l2_norm_tolerance=0.3,
    delta_l2_norm_tolerance=1.001,
)

In [ ]:
inv.invert(
    plot_dynamic_convergence=True,
    results_fname="../tmp/constraining_the_inversion",
)

In [ ]:
inv.stats_df

In [ ]:
inv.plot_inversion_results(
    iters_to_plot=2,
)

In [ ]:
_ = polar_utils.grd_compare(
    true_basement_topography,
    inv.model.topography,
    region=grav_data.inner_region,
    grid1_name="True topography",
    grid2_name="Inverted topography",
    robust=True,
    hist=True,
    inset=False,
    verbose="q",
    title="difference",
    grounding_line=False,
    reverse_cpt=True,
    cmap="rain",
    frame=True,
)

In [ ]:
# sample the inverted topography at the constraint points
constraint_points = invert4geom.sample_grids(
    constraint_points,
    inv.model.topography,
    "inverted_topography",
)

rmse_without_weighting = invert4geom.rmse(
    constraint_points.upward - constraint_points.inverted_topography
)
max_error_without_weighting = vd.maxabs(
    constraint_points.upward - constraint_points.inverted_topography
)

print(f"RMSE at constraints: {round(rmse_without_weighting, 1)} m")
print(f"max error at constraints: {round(max_error_without_weighting, 1)} m")